In [2]:
import time
import lattice_challenges_01

# in this notebook's comments, "triangular matrix"
#   should always have the meaning of "lower triangular matrix"

In [1]:
# this function returns the coefficient in Gram-Schmidt
#   orthogonalization; the orthogonalized vector is v_i;
#   the vector by which it is improved is v_j
def gs_mu(v_i, v_j):
    return (v_i*v_j)/(v_j*v_j)

# this function returns a matrix which is the result
#   of Gram-Shmidt orthogonalization process (not orthonormalization)
def gram_schmidt_orthogonalization(basis):
    ret = matrix(RR, basis.nrows(), basis.ncols())
    for i in range(basis.nrows()):
        v = basis[i]
        for j in range(i):
            v -= gs_mu(v, ret[j]) * ret[j]
        ret[i] = v
    return ret

# used in LLL (regardless of which condition is used)
def swap_rows_in_LLL(b, g, i):
    g_i_1 = g[i] + gs_mu(b[i], g[i-1])*g[i-1]
    g[i] = g[i-1] - gs_mu(b[i-1], g_i_1)*g_i_1
    g[i-1] = g_i_1
    b[i], b[i-1] = b[i-1], b[i]

# implementation of classical LLL algorithm
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (including 0.75)
#   but the algorithm terminates for delta < 1.0
def LLL_usual_condition(input_basis, delta):
    b = copy(input_basis)
    n = b.nrows()
    g = gram_schmidt_orthogonalization(b)
    i = 0
    while i < n:
        for j in range(i-1, -1, -1):
            b[i] -= round(gs_mu(b[i], g[j])) * b[j]
        if i > 0 and \
                (delta - gs_mu(b[i], g[i-1])**2)*(g[i-1]*g[i-1]) > g[i]*g[i]:
            swap_rows_in_LLL(b, g, i)
            i -= 1
        else:
            i += 1
    return b

# implementation of LLL algorithm with different condition as used in
#   https://link.springer.com/chapter/10.1007/978-3-319-94821-8_10
# input_basis is the matrix to be reduced;
#   delta should be a number between 0.25 and 0.75 (not including 0.75)
def LLL_unusual_condition(input_basis, delta):
    b = copy(input_basis)
    n = b.nrows()
    g = gram_schmidt_orthogonalization(b)
    i = 0
    while i < n:
        for j in range(i-1, -1, -1):
            b[i] -= round(gs_mu(b[i], g[j])) * b[j]
        if i > 0 and delta*(g[i-1]*g[i-1]) > g[i]*g[i]:
            swap_rows_in_LLL(b, g, i)
            i -= 1
        else:
            i += 1
    return b

# returns A * A^T
# input must be integer matrix
def gram_matrix(A):
    ret = matrix(ZZ, A.nrows(), A.nrows())
    for i in range(A.nrows()):
        for j in range(i):
            ret[i, j] = A[i]*A[j]
            ret[j, i] = ret[i, j]
    for k in range(A.nrows()):
        ret[k, k] = A[k]*A[k]
    return ret

def gram_matrix_RR(A):
    ret = matrix(RR, A.nrows(), A.nrows())
    for i in range(A.nrows()):
        for j in range(i):
            ret[i, j] = A[i]*A[j]
            ret[j, i] = ret[i, j]
    for k in range(A.nrows()):
        ret[k, k] = A[k]*A[k]
    return ret

# Cholesky decomposition of G (positive definite quadratic form)
def cholesky_decomposition(G):
    rows = G.nrows()
    ret = matrix(RR, rows, rows)
    for m in range(rows):
        for n in range(m):
            t = 0  # temporary sum
            for i in range(n):
                t += ret[m,i]*ret[n,i]
            ret[m,n] = (G[m,n]-t)/ret[n,n]
        t = 0
        for i in range(m):
            t += ret[m,i]**2
        ret[m,m] = sqrt(G[m,m]-t)
        # the preceding expression can also be taken with negative sign
    return ret

# returns a triangular matrix having the same Gramian matrix
def triangularize_basis(b):
    return cholesky_decomposition(gram_matrix(b))

# this function reduces the input tr_basis from bottom
# performs the same changes on parallel_basis
# the function has side effect on both input parameters
# the return value indicates whether any change of the input
#   parameters has been made (is True if so)
def reduce_triangular_basis(tr_basis, parallel_basis):
    change = False
    for c in range(tr_basis.nrows()-1, -1, -1):
        for m in range(c+1, tr_basis.nrows()):
            t = round(tr_basis[m, c]/tr_basis[c, c])
            if t != 0:
                change = True
                tr_basis[m] -= t*tr_basis[c]
                parallel_basis[m] -= t*parallel_basis[c]
    return change

# similar effect as reduce_triangular_basis above
def reduce_triangular_basis_from_top(tr_basis, parallel_basis):
    change = False
    for m in range(1, tr_basis.nrows()):
        for c in range(m-1, -1, -1):
            t = round(tr_basis[m, c]/tr_basis[c, c])
            if t != 0:
                change = True
                tr_basis[m] -= t*tr_basis[c]
                parallel_basis[m] -= t*parallel_basis[c]
    return change

# returns the sum of natural logarithms of the elements of the input alist
def sum_of_logarithms(alist):
    ret = 0.0
    for val in alist:
        ret += float(log(val))
    return ret

# sorts input b via side effect; returns a list of squared euclidean norms
def sort_by_norms(b):
    squared_norms = [(b[i]*b[i], i) for i in range(b.nrows())]
    squared_norms.sort()
    perm_dict = {}
    for i in range(len(squared_norms)):
        perm_dict[i] = squared_norms[i][1]
    while perm_dict:
        start_index, next_index = perm_dict.popitem()
        start_row = b[start_index]
        cur_index = start_index
        while next_index != start_index:
            b[cur_index] = b[next_index]
            cur_index = next_index
            next_index = perm_dict.pop(next_index)
        b[cur_index] = start_row
    return [squared_norms[i][0] for i in range(b.nrows())]

# sort by euclidean norms and reduce as long
#   as sum of logarithms gets lower
# works with a copy of input_basis; does not have side effects
def reduce_basis_using_triangular_new(input_basis):
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        previous = copy(basis)
        tr_basis = triangularize_basis(basis)
        if not reduce_triangular_basis(tr_basis, basis):
            return previous
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            return previous
        previous_value = t

# similar effect as reduce_basis_using_triangular_new above
def reduce_basis_using_triangular_new_from_top(input_basis):
    basis = copy(input_basis)
    previous_value = sum_of_logarithms(sort_by_norms(basis))
    while True:
        previous = copy(basis)
        tr_basis = triangularize_basis(basis)
        if not reduce_triangular_basis_from_top(tr_basis, basis):
            return previous
        t = sum_of_logarithms(sort_by_norms(basis))
        if t >= previous_value:
            return previous
        previous_value = t

# computes some properties of input_matrix using euclidean norm
def euclidean_properties(input_matrix):
    shortest_vector_norm = math.inf
    longest_vector_norm = 0.0
    sum_of_norms = 0.0
    sum_of_logarithms_of_norms = 0.0  # natural log
    for i in range(input_matrix.nrows()):
        t = 0.0
        for j in range(input_matrix.ncols()):
            t += input_matrix[i, j]**2
        r = sqrt(t)
        sum_of_norms += r
        sum_of_logarithms_of_norms += float(log(r))
        if r > longest_vector_norm:
            longest_vector_norm = r
        if r < shortest_vector_norm:
            shortest_vector_norm = r
    return (shortest_vector_norm, longest_vector_norm,
            sum_of_norms, sum_of_logarithms_of_norms)

# sort by euclidean norms and reduce as long
#   as sum of logarithms gets lower
# works with a copy of input_basis; does not have side effects
def reduce_basis_using_triangular_new_A(input_basis):
    B = copy(input_basis)
    old = euclidean_properties(B)[3]*2
    cur = euclidean_properties(B)[3]
    while cur < old:
        old = cur
        B = reduce_basis_using_triangular_new(B)
        B = reduce_basis_using_triangular_new_from_top(B)
        cur = euclidean_properties(B)[3]
    return B

# similar effect as reduce_basis_using_triangular_new_A above
def reduce_basis_using_triangular_new_B(input_basis):
    B = copy(input_basis)
    old = euclidean_properties(B)[3]*2
    cur = euclidean_properties(B)[3]
    while cur < old:
        old = cur
        B = reduce_basis_using_triangular_new_from_top(B)
        B = reduce_basis_using_triangular_new(B)
        cur = euclidean_properties(B)[3]
    return B

# in triangular matrix, this function changes row (vector)
#   at index line_index in input_matrix by subtracting row
#   at index by_index k times
def sub_in_tr(input_matrix, line_index, by_index, k):
    for i in range(by_index+1):
        input_matrix[line_index, i] -= k*input_matrix[by_index, i]

# performs changes in triangular matrix (tr) corresponding to swaping
#   rows at indices i and i-1 in the corresponding basis, so that
#   gramian matrices of tr and the corresponding basis remain the same
def swap_rows_in_tr(tr, i):
    bk = tr[i, i-1]
    ak = tr[i-1, i-1]
    bk1 = tr[i, i]
    tr[i, i-1] = sqrt(bk**2+bk1**2)
    tr[i-1, i-1] = ak*bk/tr[i, i-1]
    tr[i-1, i] = ak*bk1/tr[i, i-1]
    tr[i, i] = 0
    tr[i], tr[i-1] = tr[i-1], tr[i]
    for x in range(i+1, tr.nrows()):
        xa = tr[x, i-1]
        tr[x, i-1] = (xa*bk+tr[x, i]*bk1)/tr[i-1, i-1]
        tr[x, i] = (xa*ak-tr[x, i-1]*tr[i, i-1])/tr[i, i]

# faster implementation of classical LLL
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_usual_condition_using_triangular(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    i = 0
    flag = False
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * delta > tr[i, i]**2 + tr[i, i-1]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis

# faster implementation of LLL with different condition
# uses triangular matrix instead of Gram-Schmidt orthogonalization
# does not perform for-cycle after i has been decreased
def LLL_unusual_condition_using_triangular(input_basis, delta):
    sqrt_delta = sqrt(delta)
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    i = 0
    flag = False
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]*sqrt_delta > tr[i, i]:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis


In [2]:
# the rank of the input basis must be equal to .nrows()
# assumed the inputs are integer matrix and integer vector
#   of the same dimension (row length)
def check_vector_in_lattice(basis, a_vector):
    return basis.solve_left(a_vector).denominator() == 1

# assumed the inputs are integer matrices of the same dimension
# for matrices with linearly dependant rows the returned value
#   can possibly be False even if tested_matrix is a correct
#   basis defining the same lattice as original_basis
def check_reduction_correctness(original_basis, tested_matrix):
    ret = True
    for a_row in tested_matrix:
        if not check_vector_in_lattice(original_basis, a_row):
            return False
    return original_basis.rank()==tested_matrix.rank()


In [3]:
# adds the_matrix to file as a python function
# creates or modifies a file
def add_matrix_to_file(filename, matrixname, the_matrix):
    f = open(filename, 'a')
    f.write("\ndef "+str(matrixname)+"():\n    return [")
    for i in range(the_matrix.nrows()-1):
        f.write(str(list(the_matrix[i]))+",\n            ")
    if the_matrix.nrows()>0:
        f.write(str(list(the_matrix[the_matrix.nrows()-1])))
    f.write("]\n")
    f.close()


In [4]:
# generates all k-element subsets of {0, 1, ..., n-1} as ordered lists
def k_elem_subsets(n, k):
    indices = list(range(k))
    while True:
        yield indices[:]
        i = -1
        while i>=-k and indices[i]==n+i:
            i -= 1
        if i<-k:
            break
        val = indices[i]
        for j in range(i, 0):
            val += 1
            indices[j] = val

# auxiliary generator used in ceils_and_floors
def ceils_and_floors_aux(alist, acc, c):
    if c==len(alist):
        yield acc[:]
    else:
        acc[c] = floor(alist[c])
        yield from ceils_and_floors_aux(alist, acc, c+1)
        t = ceil(alist[c])
        if t != acc[c]:
            acc[c] = t
            yield from ceils_and_floors_aux(alist, acc, c+1)

# generates all possible combinations (as lists) of ceils and floors
#   of the input seq
def ceils_and_floors(seq):
    alist = list(seq)
    yield from ceils_and_floors_aux(alist, [0]*len(alist), 0)


In [5]:
# G ... Gramian matrix
# index ... index of the reduced row
# indices ... list of indices of rows by which the
#             index-th row is being reduced
# returns a list of non-integer coefficients defining
#   how many times the rows should be added
def gram_and_solve_find_min(G, indices, index):
    d = len(indices)
    left_side = matrix(ZZ, d, d)
    right_side = [0]*d
    for i in range(d):
        for j in range(d):
            left_side[i, j] = G[indices[i], indices[j]]
        right_side[i] = -G[indices[i], index]
    return list(left_side.solve_right(vector(right_side)))

# reduces each row by each possible subset of other rows
#   as long as anything changes
def gram_and_solve_reduction(basis):
    B = copy(basis)
    G, n, k = gram_matrix(B), B.nrows(), 2
    while k<=n:
        reset = False
        for indices in k_elem_subsets(n, k):
            for index in indices:
                alist = gram_and_solve_find_min(G, indices[:index]+indices[index+1:], index)
                min_vec, min_norm, min_comb = vector([0]*B.ncols()), math.inf, vector([0]*k)
                for comb in ceils_and_floors(alist[:index]+[1]+alist[index:]):
                    tvec, avec = vector(comb), vector(ZZ, B.ncols())
                    for i in range(k):
                        avec += B[indices[i]]*tvec[i]
                    t = avec*avec
                    if t < min_norm:
                        min_norm, min_vec, min_comb = t, avec, tvec
                if min_norm < B[index]*B[index]:
                    B[index], bvec = min_vec, vector(ZZ, n)
                    for i in range(k):
                        bvec += G[indices[i]]*min_comb[i]
                    G[index] = bvec
                    for i in range(n):
                        G[i, index] = G[index, i]
                    c = 0
                    for i in range(k):
                        c += G[index, indices[i]]*min_comb[i]
                    G[index, index], k, reset = c, 1, True
                    break
            if reset:
                break
        k += 1
    return B

# reduces each row by the other rows
#   as long as anything changes
def gram_and_solve_simplified(basis):
    B = copy(basis)
    G, n, indices, change = gram_matrix(B), B.nrows(), list(range(B.nrows())), True
    m = n  # m is index of the most recently changed row
    while change:
        change = False
        for index in range(n):
            if index == m:
                break
            alist = gram_and_solve_find_min(G, indices[:index]+indices[index+1:], index)
            min_vec, min_norm, min_comb = vector([0]*B.ncols()), math.inf, vector([0]*n)
            for comb in ceils_and_floors(alist[:index]+[1]+alist[index:]):
                tvec, avec = vector(comb), vector(ZZ, B.ncols())
                for i in range(n):
                    avec += B[i]*tvec[i]
                t = avec*avec
                if t < min_norm:
                    min_norm, min_vec, min_comb = t, avec, tvec
            if min_norm < B[index]*B[index]:
                B[index], bvec = min_vec, vector(ZZ, n)
                for i in range(n):
                    bvec += G[i]*min_comb[i]
                G[index] = bvec
                for i in range(n):
                    G[i, index] = G[index, i]
                c = 0
                for i in range(n):
                    c += G[index, i]*min_comb[i]
                G[index, index], change, m = c, True, index
    return B


In [6]:
def increment_in_enumeration(inr_bool, k, i):
    if inr_bool[i]:
        k[i] = (-k[i]+1 if k[i]<=0 else -k[i])
    else:
        k[i] = (-k[i]-1 if k[i]>=0 else -k[i])
    

# basis ... input basis (should be linearly independent)
# R ... radius (the length of the shortest vector in basis implicitly)
# returns the shortest nonzero vector inside a ball of given radius
def enumeration_SVP_using_triangular(basis, R=None):
    n, G = basis.nrows(), gram_matrix(basis)
    s2, i_cur_min = G[0, 0], 0
    for i in range(1, n):
        if G[i, i] < s2:
            i_cur_min, s2 = i, G[i, i]
    s_comb = [(1 if i==i_cur_min else 0) for i in range(n)]
    if R is None:
        R2 = s2
    else:
        R2 = R*R
        s2 = min(R2, s2)
    b, x, q, p, k, r, t, i = cholesky_decomposition(G), [0]*n, [0.0]*n, [0.0]*n, [0]*n, [0.0]*n, [0]*n, n-1
    inr_bool = [True]*n  # True for k[i] ranging over 0, 1, -1, 2, -2, ...
    while i<n:
        if i==0:
            x[0] = t[0] + k[0]
            q_i_1 = q[0]+(p[0]+b[0, 0]*x[0])**2
            if q_i_1 > 0.0 and q_i_1 < s2:
                s2, s_comb = q_i_1, x[:]
                increment_in_enumeration(inr_bool, k, 0)
                continue
            k[0], i = 0, 1
            continue
        x[i] = t[i] + k[i]
        q[i-1] = q[i]+(p[i]+b[i,i]*x[i])**2
        if q[i-1] <= s2:
            p[i-1] = b[i, i-1]*x[i]+r[i]
            a = -p[i-1]/b[i-1, i-1]
            t[i-1] = round(a)
            inr_bool[i-1] = (t[i-1]==math.floor(a))
            increment_in_enumeration(inr_bool, k, i)
            if i>1:
                r[i-1] = sum([b[j, i-2]*x[j] for j in range(i, n)])
            i -= 1
        else:
            k[i] = 0
            i += 1
    ret = vector(s_comb)*basis
    if ret*ret <= R2:
        return ret
    return None

def linearly_independent(u, v):
    uvec, vvec = vector(u), vector(v)
    return (uvec*uvec)*(vvec*vvec) - (uvec*vvec)**2

def enumeration_SVP_using_triangular_for_two_vectors(basis):
    n, G = basis.nrows(), gram_matrix(basis)
    if G[0, 0] <= G[1,1]:
        aa, bb = 0, 1
    else:
        aa, bb = 1, 0
    st2, i_cur_st = G[aa, aa], aa
    nd2, i_cur_nd = G[bb, bb], bb
    for i in range(1, n):
        if G[i, i] < st2:
            i_cur_nd, nd2 = i_cur_st, st2
            i_cur_st, st2 = i, G[i, i]
        elif G[i, i] < nd2:
            i_cur_nd, nd2 = i, G[i, i]
    st_comb = [(1 if i==i_cur_st else 0) for i in range(n)]
    nd_comb = [(1 if i==i_cur_nd else 0) for i in range(n)]
    b, x, q, p, k, r, t, i = cholesky_decomposition(G), [0]*n, [0.0]*n, [0.0]*n, [0]*n, [0.0]*n, [0]*n, n-1
    inr_bool = [True]*n  # True for k[i] ranging over 0, 1, -1, 2, -2, ...
    while i<n:
        if i==0:
            x[0] = t[0] + k[0]
            q_i_1 = q[0]+(p[0]+b[0, 0]*x[0])**2
            if q_i_1 > 0.0 and q_i_1 < nd2:
                new_comb = x[:]
                if q_i_1 < st2:
                    if linearly_independent(new_comb, st_comb):
                        nd2, nd_comb = st2, st_comb
                    st2, st_comb = q_i_1, new_comb
                elif q_i_1 < nd2 and linearly_independent(new_comb, st_comb):
                    nd2, nd_comb = q_i_1, new_comb
                increment_in_enumeration(inr_bool, k, 0)
                continue
            k[0], i = 0, 1
            continue
        x[i] = t[i] + k[i]
        q[i-1] = q[i]+(p[i]+b[i,i]*x[i])**2
        if q[i-1] <= nd2:
            p[i-1] = b[i, i-1]*x[i]+r[i]
            a = -p[i-1]/b[i-1, i-1]
            t[i-1] = round(a)
            inr_bool[i-1] = (t[i-1]==math.floor(a))
            increment_in_enumeration(inr_bool, k, i)
            if i>1:
                r[i-1] = sum([b[j, i-2]*x[j] for j in range(i, n)])
            i -= 1
        else:
            k[i] = 0
            i += 1
    return vector(st_comb)*basis, vector(nd_comb)*basis

def enumeration_CVP_using_triangular(basis, u):
    n, G = basis.nrows()+1, gram_matrix(basis)
    s2, i_cur_min = (basis[0]-u)*(basis[0]-u), 0
    for i in range(1, n-1):
        if (basis[i]-u)*(basis[i]-u) < s2:
            i_cur_min, s2 = i, (basis[i]-u)*(basis[i]-u)
    s_comb = [(1 if i==i_cur_min else 0) for i in range(n-1)]
    temp = cholesky_decomposition(G)
    b = matrix(RR, n, n-1)
    for i in range(n-1):
        b[i] = temp[i]
    b[n-1] = u*((temp.inverse()*basis).transpose())
    x, q, p, k, r, t, i = [0]*n, [0.0]*n, [0.0]*n, [0]*n, [0.0]*n, [0]*n, n-1
    inr_bool = [True]*n  # True for k[i] ranging over 0, 1, -1, 2, -2, ...
    x[i] = -1
    p[i-1] = b[i, i-1]*x[i]
    a = -p[i-1]/b[i-1, i-1]
    t[i-1] = round(a)
    inr_bool[i-1] = (t[i-1]==math.floor(a))
    r[i-1] = b[i, i-2]*x[i]
    i -= 1
    while i<n-1:
        if i==0:
            x[0] = t[0] + k[0]
            q_i_1 = q[0]+(p[0]+b[0, 0]*x[0])**2
            if q_i_1 < s2:
                s2, s_comb = q_i_1, x[:n-1]
                increment_in_enumeration(inr_bool, k, 0)
                continue
            k[0], i = 0, 1
            continue
        x[i] = t[i] + k[i]
        q[i-1] = q[i]+(p[i]+b[i,i]*x[i])**2
        if q[i-1] <= s2:
            p[i-1] = b[i, i-1]*x[i]+r[i]
            a = -p[i-1]/b[i-1, i-1]
            t[i-1] = round(a)
            inr_bool[i-1] = (t[i-1]==math.floor(a))
            increment_in_enumeration(inr_bool, k, i)
            if i>1:
                r[i-1] = sum([b[j, i-2]*x[j] for j in range(i, n)])
            i -= 1
        else:
            k[i] = 0
            i += 1
    return vector(s_comb)*basis


In [7]:
# on this matrix, LLL does not find the shortest vector
#   while gramAndSolve and enumeration do
# (from https://crypto.stackexchange.com/questions/37196/find-an-example-of-a-lattice-such-that-lll-algorithm-cant-find-the-shortest-vec)
L = matrix([[16252, 5541, 48769, 2593, 22395],
            [2404, 46418, 5335, 59631, 29390],
            [41816, 27475, 56363, 50417, 62371],
            [27419, 53783, 1412, 42344, 55983],
            [62503, 52719, 22160, 3940, 64256]])

# on this matrix, the enumeration runs unusually long (even longer than gram_and_solve_simplified)
# found randomly
A = matrix([[39450, -1, -51965, 37376, 54699, 62019, -24917, -56741, -37148, 30791, -61252, 56546],
            [28213, 18923, -39890, -12599, 54760, -29860, 37255, 52830, 56208, 40008, 40911, -28483],
            [-6979, 25394, 28906, -16173, -21753, -62430, 36910, -1379, -62922, 6759, -40726, -1222],
            [51019, 26934, 56685, 5662, 28091, -64399, 3264, 59083, -31035, 49120, -10801, 56806],
            [-34891, -45174, -1956, 52203, 27533, 63705, -62060, -62047, 55565, 58240, 19340, -10976],
            [-9006, -6421, -62272, -62015, 47794, 10658, -27132, -15272, -16756, 48073, 49068, 23670],
            [6825, 49025, 15065, 53684, 57899, -16880, -52560, 43522, 54587, -21981, -64346, -5357],
            [-34387, -16659, -31467, 24034, 59394, -61422, 49348, -33197, 8477, 12763, -10272, -2929],
            [60419, 27900, 35501, -4033, 34752, -37807, -50648, 23311, -13503, 60911, -18204, 46778],
            [45047, 45636, -29536, 14926, -33075, -510, -26142, -4026, -56696, -9264, -25461, 47085],
            [13658, -18002, 51372, -28904, -43333, -47794, 23066, 15399, 37600, 54859, 65516, -51652],
            [51022, 62799, 16837, 12314, -39644, -31050, -54361, 13868, 34833, 63125, -15206, -34825]])

In [8]:
def complete_row_to_unimodular_old(v):
    n = len(v)
    r = [0]*(n-1)
    s = [0]*(n-1)
    t = [0]*(n-1)
    r[0] = v[0]
    for i in range(1,n-1):
        r[i] = GCD(r[i-1], v[i])
    if GCD(r[n-2], v[n-1])!=1:
        return None
    for i in range(n-2):
        s[i] = r[i]//r[i+1]
    s[n-2] = r[n-2]
    for i in range(n-2):
        t[i] = v[i+1]//r[i+1]
    t[n-2] = v[n-1]
    ret = matrix(ZZ, n, n)
    ret[0,0] = 1
    for i in range(1,n):
        tup = xgcd(t[i-1], s[i-1])
        ret[i,i] = tup[2]
        for j in range(i):
            ret[i,j] = -tup[1]*ret[0,j]
            ret[0,j] *= s[i-1]
        ret[0,i] = t[i-1]
    return ret

# returned matrix will not be unimodular
#   if v is not primitive (coprime)
def complete_row_to_unimodular(v):
    n = len(v)
    ret = matrix(ZZ, n, n)
    ret[0] = v
    d = v[0]
    for i in range(1,n):
        tup = xgcd(d, v[i])
        ret[i,i] = tup[1]
        for j in range(i):
            ret[i,j] = -tup[2]*ret[0,j]//d
        d = tup[0]
    return ret


#v = vector([18,96,66,39,26])
v = vector([18,96,66,38,26])
#M = complete_row_to_unimodular_old(v)
#print(M)
#print(det(M))
#N = complete_row_to_unimodular(v)
#print(N)
#print(det(N))

In [11]:
def LLL_usual_condition_using_triangular_with_prints(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    last = copy(tr)
    last[0,0] += 1
    i = 0
    flag = True
    while i < number_of_vectors:
        if last != tr:
            # print(tr)
            print(basis)
            print()
            last = copy(tr)
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * \
                   (delta-(tr[i, i-1]/tr[i-1, i-1])**2) > tr[i, i]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            print("swapped at " + str(i))
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    if last != tr:
        # print(tr)
        print(basis)
        print()
        last = copy(tr)
    return basis

n = 29
C = matrix([[8*n,0,0,0,0],[0,8*n,0,0,0],[0,0,8*n,0,0],
            [8*5, 8*6, 8*27, 1, 0],
            [8*8, 8*23, 8*1, 0, n]])
output = LLL_usual_condition_using_triangular_with_prints(C, 0.75)
print(list(map(lambda v: float(sqrt(v*v)), output)))


[232   0   0   0   0]
[  0 232   0   0   0]
[  0   0 232   0   0]
[ 40  48 216   1   0]
[ 64 184   8   0  29]

swapped at 3
[232   0   0   0   0]
[  0 232   0   0   0]
[ 40  48 -16   1   0]
[  0   0 232   0   0]
[ 64 184   8   0  29]

swapped at 2
[232   0   0   0   0]
[ 40  48 -16   1   0]
[  0 232   0   0   0]
[  0   0 232   0   0]
[ 64 184   8   0  29]

swapped at 1
[ 40  48 -16   1   0]
[232   0   0   0   0]
[  0 232   0   0   0]
[  0   0 232   0   0]
[ 64 184   8   0  29]

[ 40  48 -16   1   0]
[152 -96  32  -2   0]
[  0 232   0   0   0]
[  0   0 232   0   0]
[ 64 184   8   0  29]

swapped at 2
[ 40  48 -16   1   0]
[ 32  -8  80  -5   0]
[152 -96  32  -2   0]
[  0   0 232   0   0]
[ 64 184   8   0  29]

[ 40  48 -16   1   0]
[ 32  -8  80  -5   0]
[120 -88 -48   3   0]
[  0   0 232   0   0]
[ 64 184   8   0  29]

swapped at 3
[ 40  48 -16   1   0]
[ 32  -8  80  -5   0]
[-24  64  56  11   0]
[120 -88 -48   3   0]
[ 64 184   8   0  29]

swapped at 3
[ 40  48 -16   1   0]
[ 32  -8  80

In [10]:
n = 17
C = matrix([[8*n,0,0,0,0],[0,8*n,0,0,0],[0,0,8*n,0,0],
            [8*10, 8*8, 8*2, 1, 0],
            [8*10, 8*0, 8*9, 0, n]])
output = LLL_usual_condition_using_triangular_with_prints(C, 0.75)
print(list(map(lambda v: float(sqrt(v*v)), output)))


[136   0   0   0   0]
[  0 136   0   0   0]
[  0   0 136   0   0]
[ 80  64  16   1   0]
[ 80   0  72   0  17]

swapped at 3
[136   0   0   0   0]
[  0 136   0   0   0]
[-56  64  16   1   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

swapped at 2
[136   0   0   0   0]
[-56  64  16   1   0]
[  0 136   0   0   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

swapped at 1
[-56  64  16   1   0]
[136   0   0   0   0]
[  0 136   0   0   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

[-56  64  16   1   0]
[ 80  64  16   1   0]
[  0 136   0   0   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

swapped at 2
[-56  64  16   1   0]
[-24   8 -32  -2   0]
[ 80  64  16   1   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

swapped at 1
[-24   8 -32  -2   0]
[-56  64  16   1   0]
[ 80  64  16   1   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

[-24   8 -32  -2   0]
[-32  56  48   3   0]
[ 80  64  16   1   0]
[  0   0 136   0   0]
[ 80   0  72   0  17]

[-24   8 -32  -2   0]
[-32  56  48   3   0]
[ 5

In [11]:
n = 127
C = matrix([[1016, 0, 0, 0, 0, 0], [0, 1016, 0, 0, 0, 0], [0, 0, 1016, 0, 0, 0], [0, 0, 0, 1016, 0, 0], [280, 968, 40, 608, 1, 0], [472, 136, 400, 616, 0, 127]])
M = LLL_usual_condition_using_triangular(C, 0.99)
print(M)
N = copy(M)
N[0], N[-1] = N[-1], N[0]
print()
Z = LLL_usual_condition_using_triangular(N, 0.99)
print(Z)

enumeration_SVP_using_triangular_for_two_vectors(C)


[   0    0    0    0  127    0]
[ 112  184   16   40  -25    0]
[  -8  -72 -104 -136  -38  127]
[ -64   40  136 -168  -22    0]
[ 144  -40  208  136   46  127]
[ 240 -200   -8 -144  -41 -127]

[   0    0    0    0  127    0]
[ 112  184   16   40  -25    0]
[  -8  -72 -104 -136  -38  127]
[ -64   40  136 -168  -22    0]
[ 144  -40  208  136   46  127]
[ 240 -200   -8 -144  -41 -127]


((0, 0, 0, 0, 127, 0), (-112, -184, -16, -40, 25, 0))

In [12]:
n = 127
C = matrix([[1016, 0, 0, 0, 0, 0], [0, 1016, 0, 0, 0, 0], [0, 0, 1016, 0, 0, 0], [0, 0, 0, 1016, 0, 0], [816, 392, 456, 832, 1, 0], [456, 120, 328, 120, 0, 127]])
M = LLL_usual_condition_using_triangular(C, 0.99)
print(M)
print(enumeration_SVP_using_triangular_for_two_vectors(C))
print()

U = complete_row_to_unimodular(vector([102, 49, 57, 104, -127, 0]))
print(U)
print()
print(C)
print()
print(U*C)
print(check_reduction_correctness(C, U*C))
print(enumeration_SVP_using_triangular_for_two_vectors(U*C))
#print((U*C).LLL(delta=0.99))
print((U*C).solve_left(vector([-64, -80, -112, 48, 28, 127])))

[   0    0    0    0 -127    0]
[ -64  -80 -112   48   28  127]
[  16  -72  248   96    5    0]
[-152  176  184  104   16    0]
[ 192  152  -72  136   60    0]
[ 184  -40   40  -64  -37  254]
((0, 0, 0, 0, 127, 0), (-64, -80, -112, 48, 28, 127))

[ 102   49   57  104 -127    0]
[ -25  -12    0    0    0    0]
[   0    0    1    0    0    0]
[   0    0    0    1    0    0]
[   0    0    0    0    1    0]
[   0    0    0    0    0    1]

[1016    0    0    0    0    0]
[   0 1016    0    0    0    0]
[   0    0 1016    0    0    0]
[   0    0    0 1016    0    0]
[ 816  392  456  832    1    0]
[ 456  120  328  120    0  127]

[     0      0      0      0   -127      0]
[-25400 -12192      0      0      0      0]
[     0      0   1016      0      0      0]
[     0      0      0   1016      0      0]
[   816    392    456    832      1      0]
[   456    120    328    120      0    127]
True
((0, 0, 0, 0, -127, 0), (-64, -80, -112, 48, 28, 127))
(1, 5, -70, -127, 155, 1)


In [13]:
C = matrix([[508, 0, 0, 0, 0, 0, 0, 0, 0], [0, 508, 0, 0, 0, 0, 0, 0, 0], [0, 0, 508, 0, 0, 0, 0, 0, 0], [0, 0, 0, 508, 0, 0, 0, 0, 0], [0, 0, 0, 0, 508, 0, 0, 0, 0], [0, 0, 0, 0, 0, 508, 0, 0, 0], [0, 0, 0, 0, 0, 0, 508, 0, 0], [20, 496, 328, 376, 292, 260, 220, 1, 0], [388, 392, 160, 60, 312, 296, 144, 0, 127]])
M = LLL_usual_condition_using_triangular(C, 0.99)
print(M)
print(enumeration_CVP_using_triangular(C[range(C.nrows()-1), range(C.ncols()-1)], C[C.nrows()-1][:C.ncols()-1]))
#print(gram_and_solve_reduction(C))
print(gram_and_solve_simplified(C))

[   0    0    0    0    0    0    0 -127    0]
[  12    8  -12 -100    4  -20   72   32  127]
[   4 -104  -36 -128  160   52   44   51    0]
[ 104   24  -16  -56   64    4  188   50 -127]
[ -88  -64  188   -8    4   48  108   15 -127]
[ -36  -80 -184  136   84   40  112   49    0]
[  40  164   52  -40   44  188   -8   -4 -127]
[ -52   16 -136 -144  -80    8   -4  -34 -127]
[  48 -232   76  -12 -112  116   20  -23    0]
(376, 384, 172, 160, 308, 316, 72, -32)
[  48 -232   76  -12 -112  116   20  -23    0]
[  -8 -112  -24  -28  156   72  -28   19 -127]
[  40  164   52  -40   44  188   -8   -4 -127]
[ -88  -64  188   -8    4   48  108   15 -127]
[  52  -16  136  144   80   -8    4   34  127]
[ -36  -80 -184  136   84   40  112   49    0]
[ -12   -8   12  100   -4   20  -72  -32 -127]
[   0    0    0    0    0    0    0 -127    0]
[ 104   24  -16  -56   64    4  188   50 -127]


In [14]:
def LLL_usual_condition_using_triangular_with_prints_G(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = triangularize_basis(basis)
    last = copy(tr)
    last[0,0] += 1
    i = 0
    flag = True
    while i < number_of_vectors:
        if last != tr:
            print(tr*tr.transpose())
            print()
            last = copy(tr)
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * \
                   (delta-(tr[i, i-1]/tr[i-1, i-1])**2) > tr[i, i]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    if last != tr:
        print(tr*tr.transpose())
        print()
        last = copy(tr)
    return basis

n = 65537
C = matrix([[8*n,0,0,0,0],[0,32*n,0,0,0],[0,0,64*n,0,0],
            [8*(-12), 32*7, 64*11, 1, 0],
            [8*4, 32*(-9), 64*(-5), 0, n]])
LLL_usual_condition_using_triangular_with_prints_G(C, 0.75)


[2.74886295616000e11   0.000000000000000   0.000000000000000 -5.03324160000000e7  1.67774720000000e7]
[  0.000000000000000 4.39818072985600e12   0.000000000000000  4.69769216000000e8 -6.03988992000000e8]
[  0.000000000000000   0.000000000000000 1.75927229194240e13  2.95283507200000e9 -1.34219776000000e9]
[-5.03324160000000e7  4.69769216000000e8  2.95283507200000e9    555009.000000000   -292864.000000000]
[ 1.67774720000000e7 -6.03988992000000e8 -1.34219776000000e9   -292864.000000000  4.29528473700000e9]

[2.74886295616000e11   0.000000000000000 -5.03324160000000e7   0.000000000000000  1.67774720000000e7]
[  0.000000000000000 4.39818072985600e12  4.69769216000000e8   0.000000000000000 -6.03988992000000e8]
[-5.03324160000000e7  4.69769216000000e8    555009.000000000  2.95283507200000e9   -292864.000000000]
[  0.000000000000000   0.000000000000000  2.95283507200000e9 1.75927229194240e13 -1.34219776000000e9]
[ 1.67774720000000e7 -6.03988992000000e8   -292864.000000000 -1.34219776000000e9 

[    -96     224     704       1       0]
[      0       0       0   65537       0]
[    -64     -64     384       1   65537]
[-148008 -178944   36800   17926       0]
[ 367552 -158560  100864   18017       0]

In [12]:
matice = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1]])
matice2 = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1]])
print(enumeration_CVP_using_triangular(matrix([[2,3,4],[5,6,7]]), vector([2,2,2])))
print(enumeration_CVP_using_triangular(matice[:25], matice[25]))
print(enumeration_CVP_using_triangular(matice[1:], matice[0]))
print(enumeration_SVP_using_triangular(matice))

matice3 = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1]])
print(enumeration_CVP_using_triangular(matice3[:24], matice3[24]))
print(enumeration_CVP_using_triangular(matice3[1:], matice3[0]))
print(enumeration_SVP_using_triangular(matice3))


(3, 3, 3)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)
(16, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, 2)
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5)


In [13]:
print((matice3[0]-vector([16, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, 2]))*(matice3[0]-vector([16, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, -4, 2])))
print(matice3[0]*matice3[0])

388
400


In [14]:
#C = matrix([[508, 0, 0, 0, 0, 0, 0, 0, 0], [0, 508, 0, 0, 0, 0, 0, 0, 0], [0, 0, 508, 0, 0, 0, 0, 0, 0], [0, 0, 0, 508, 0, 0, 0, 0, 0], [0, 0, 0, 0, 508, 0, 0, 0, 0], [0, 0, 0, 0, 0, 508, 0, 0, 0], [0, 0, 0, 0, 0, 0, 508, 0, 0], [20, 496, 328, 376, 292, 260, 220, 1, 0], [388, 392, 160, 60, 312, 296, 144, 0, 127]])
#C = matrix([[1016, 0, 0, 0, 0, 0], [0, 1016, 0, 0, 0, 0], [0, 0, 1016, 0, 0, 0], [0, 0, 0, 1016, 0, 0], [280, 968, 40, 608, 1, 0], [472, 136, 400, 616, 0, 127]])
#C = matrix([[8*17,0,0,0,0],[0,8*17,0,0,0],[0,0,8*17,0,0],[8*10, 8*8, 8*2, 1, 0],[8*10, 8*0, 8*9, 0, 17]])
C = matice
print(float((C[-1]-enumeration_CVP_using_triangular(C[:-1], C[-1])).norm() - C[-1].norm()))
print(float((C[0]-enumeration_CVP_using_triangular(C[1:], C[0])).norm() - C[0].norm()))
print(enumeration_SVP_using_triangular(C))


0.0
0.0
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5)


In [22]:
#C = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 0], [4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, -4, 4, 0, 5]])
C = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 0], [-2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, 0, 5]])
Cwithout1 = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 0], [-2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, 0, 5]])
Cwithout_2 = matrix([[20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 20, 0, 0], [-2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, -2, 2, 0, 5]])
print(float((C[-1]-enumeration_CVP_using_triangular(C[:-1], C[-1])).norm() - C[-1].norm()))
print(float((C[0]-enumeration_CVP_using_triangular(C[1:], C[0])).norm() - C[0].norm()))
print(enumeration_SVP_using_triangular(C))
print(float((C[-2]-enumeration_CVP_using_triangular(Cwithout_2, C[-2])).norm() - C[-2].norm()))
print(float((C[1]-enumeration_CVP_using_triangular(Cwithout1, C[1])).norm() - C[1].norm()))

0.0
0.0
(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0)
-16.126486167700197
0.0


In [2]:
def LLL_usual_condition_on_triangular(input_basis, delta):
    number_of_vectors = input_basis.nrows()
    basis = copy(input_basis)
    tr = matrix(RR, number_of_vectors, number_of_vectors, basis)
    i = 0
    flag = False
    while i < number_of_vectors:
        if flag:
            for j in range(i-1, -1, -1):
                t = round(tr[i, j]/tr[j, j])
                sub_in_tr(tr, i, j, t)
                basis[i] -= t*basis[j]
        if i>0 and tr[i-1, i-1]**2 * delta > tr[i, i]**2 + tr[i, i-1]**2:
            swap_rows_in_tr(tr, i)
            basis[i], basis[i-1] = basis[i-1], basis[i]
            i -= 1
            flag = False
        else:
            i += 1
            flag = True
    return basis

In [19]:
C = [[666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0], [248100281708870449582474917288706994596790249055506203758369233800697496799472, 19612043073829062502488525230770751669892639161001228882046221995845898025600, 575757047549795749089446409037935264674818107417962726963169971108462743508592, 350336550843487417902576414540471068904082832782671733394894035339795529151584, 150550993179534680454073801264636307159923885634620017883965012230138633789464, 593882065132183335707821216741558999433609302616849374133522946109830865980352, 509555391953750754177258655652967208519588262049079240281672028613966714047064, 167466004461506118662968151897540595257042923395574555463447233489543197242064, 622974640126246188343221786016583036198639176570732115457775333459200754722640, 319806381447277085414768215639319058346410369057312384748661586230479467409568, 602462483498331507971779337729400246882820687249136655400307597185182214994984, 285830763345607473178465407683029180611820182999472563880122255807010068599288, 523354411315663636389166769880431019035626396197043922182875438200503756670648, 214485682991459608853291979636226368310663517352710105586162345276199844222072, 267701118539901720978938956351123629062491709407459007321840582108876187684128, 250287610528089292863316587901324305267983417981003048977370993634567704274976, 435624742734631772907477087927109247594739684422881857877834954673354260336544, 304664820597041282941416783413246782403235084135547353403681890093683100706184, 200038344783395372680943526405373320651274697169366712105306779216426341138376, 357621177433062193076269424307102660086720472342856266474552951472121558700224, 203507477733545354836594595850641565633353708266614580225262840310408842837536, 303367732527711793612446089763058416199095698125115356788061807949437838345096, 354621339145771622114857758576902097240177960318943582922302772840305699371848, 244767646650841753610327564645074674800482859073016987200258099649866124734944, 256475647140418274169692677692003197835915576910936587434989049617582199840856, 128289336229302869291682476061801273597749622301458427993832914516838239377752, 581008413885285598053549678008042399558133961835413498800703600593011441762600, 203523737303532454965314037530053310408391603334938144466568524990430088128256, 23089321070735503348881921057629495919355008394340481142958733680985462070032, 90460303356116324634172331080461507532899708899029158124201810509613528283880, 283484702129178048514563847031518840067870348307920488459164069061481102383528, 280351111678988408246162965493588776494114345282579645755298802070399399999856, 307073255758225738141338405988174612280933028297729074915806019704470034071112, 86785407123666637477686417121335379496969841574916812336888165267345551400552, 602976920584489139258031345841553775958695656385187575423954326776830762232472, 245044242182976963352155744169914869162202472421889662381719100376298944865248, 557141135657626338025247773662988959638844261307999940756037035528109223374864, 562024370827535862263205231217437279689531214297096854020798655756218392725880, 330819208701344060078024708729054025343527628028298081907062367392996682069008, 644739574040005008710611417966968644369982314108810578422474966699023883551688, 33126748088302792911966161463158557803880956632522680330511170878188138872768, 112264772766378623809742317594828756665690740298395405800840166117433711293272, 380667980121928719797198131625457577887318635331674126759674448906261999097872, 201597075135558520393649106305334547055028869715036170367812128162275108270000, 104476714492840297461497280617468094091355394973808516249679457773448269808720, 177480255652501750770362642942039088124808342384016362787140513449924429645816, 331545108514598663328959340220340820738965128034490420618826778738745930402080, 426993758162712078911358744232414202860278275163696294951848195299184925686192, 473448287846955305061445733707388980901112181921100650403742294454566981876864, 269428557587949453632569657872986485435827626566719619520095415979633104581240, 420878701437698834761150088077564873819831627081167073532395426359833881293168, 543293896638545783843320882983623787019618910721554976916877410816571927229416, 96449189989165150733548544511814839658365754859490249075304987833060766323600, 287853810220891911925117213848636432615194661964399955452429608619543514421480, 534767830404402103293293364444028297123374127203052227046543418109656909801672, 462974154065422217787451392689366215309205603558742637680796222862581409553792, 334387583039281109074951332861459174252434106611801123360274276869811531441392, 604634389453368834833948386286643159927686353062668772252401897249326782621280, 339757025940943120657698057562535533473928988235069563025158827240538969851096, 380736752007189370575145051468346147846063052624632252852346110463644789574096, 427920358252622796895182976128032695968347646547139954457233614007280482293736, 666106581792691632043246670885964672571006011120917672577021350587576384271328, 29856221805293079832977951910602385890595625142935372058357747165359228992776, 41650296978513298204268147384197813185587838092218193262302623903845012226424, 234343302065154125064619281072804019224723278724669925601302383122493913416584, 186376495514938536379992411931316197000946988297355368296628539141995153138384, 237934244883535005709087574627485492549726234784882436448619889874543757894112, 63769377124007189225121790080461874084606005119649294105946917133387093829336, 87615728832050960588309643658681157565450220894655325027248172500382888304944, 65676995153810784700197584200036922082323486251772344331945538289842147870768, 88845352933750893762440981611048246319881053493806057506630792794993212134320, 249368775824158193500685305396490978829297677280088461826722819770420093344152, 299207516954335236875111214776051457407790205523999769286517599783872093651136, 196178217802430003907837857689170488456089692265773813600698785543880278272752, 21003958614461395640838147374742785564279776581001105960257845646957522559048, 535871489135101143501512713424073405290314662115004841654485066365705649435416, 531403869430547000552808302206478461049555299259054331752337706660222315376312, 43886979798562435908866182629697339452277184224560945308905475970165329085616, 346021988278755471084787402015137881996670003341399250505502022194484133278048, 444406867785573081196851452847787443840977462439116886785610252333020170394320, 146536371921396989848675295604316595532159572097765159052005312980766967975440, 102588528293790698869926891307335583175636531329894697180906907502560671709960, 26202690826814532275828681631719606183131952918459987798662146994309573319512, 152469595631411738097407498230140450682272804238237889220157143224011989016104, 396703623490355029467814018044943978436811943017066669470215442279811014465880, 548622569859132480394495752449780455233199156318692327017719169675069914516072, 163568415756656730423929209735448098721380265415689082184506293436996678696000, 113567743566640355358171836756835453143789677978725127371875592151898182764672, 130709378306466543085855498471165481313215782108462917487353003378503830410320, 394755514547920148310931020062937580767459646577899779825136602882781374132184, 340769899976103952184376316044580773829460242208547487945295349801969719464672, 177893850490023798831977381699747076493424168211064339741969328492199440096728, 466232044212444117147440861203038559869046399636904247126608623204175774552056, 140017809863537283202588794034071541900177467226692328979750502787884463897000, 132205387636937790885383350869864410976777682705594912926933513105778100950768, 62858009489813504252162485241240388026837361533412124863253547617935833904576, 334632160384707456924110787541024115265407345642685306555459733958265554330168, 1512064143088709377639730062252127880002571867895617964389447948755085281944, 70495492944969492679834264063485793341712007265158116588177997425106368958368, 470764240140545047890374083920757267839370826891772879154717804115262365982712, 281346779374940835862946480716478228763595379899379082270278478094256307026480, 421523814273531105211567658814951107965610811105303470164053083662800783997624, 64252311022030302242552100600984439369531387396541813917700576136888138266768, 262879671937328987896383337913690973554349374601603217447790293355590512574952, 312354501114910746351274849081994401313551793132004151778171378507256760157976, 589853970325570061449806641444256996682844835336203961486977111294759180391064, 551330988583612750207368820307609366369408405796230009565911569383209304684016, 172667860620293193718414679084029366856019567736788265763926023755974328206800, 402802373196592704518865873906808151819966857637115465678038924137544342958584, 662179328903104882758676268559036105656571888654901479839744015299562929209480, 78840653531800790269558568192591805438924452683318604130059301131574884766296, 222508650064283956323536043415430038343681560308366302971812914286998230033712, 320788597935924358398106255294899843882265728030951216589416884063355109102704, 172537850849666991408314817814623556590228982327504584984716418843821612221176, 412557325068589858482778772872912981012231599272109427433545275180835920932232, 149177944031462412362392790044311439480905834661022944968287042768364193765256, 149597007481750995312631922395631262072177658743512843095100974151068140817448, 571458616554191060658604236370937473299753865732978398913751856903858758768104, 329018649039123954100605222597323085427462765628437501009806465841788454903096, 366471903448018083784424085376810138657657067927947432823038197088954991320800, 106698791384787485152038373887718591440436845465321349914979297947736980982592, 382770199928373521843666845618054315906479296537547417805460967064123110174992, 309245058586039863887725593948958267764021696894481064257560869953888996308632, 303366536283087953117240081527777115328965778152483664513007299652423172989592, 502641899326238744246624299302258757739595144905584189748746641484402036739488, 439553129622473454620516216077810976564764470674042888370766177598330157209672, 11116540624015326693633303452197321643232819613411253943131912085201715813632, 311385570070841546141491584339699809173779076569361591507551008268663451187344, 1, 0], [453586087531999548675559887230681839429290866515790860094229198785849709333288, 157876260618292390165392620712244934220715353300058301653384409544952130328800, 455449285034371481219864736283695568679694410319550479791405682014401622001024, 224284458478953911557871914551265425514018939583582057005398634899401551039160, 73416833477262753077880557817453234841508866229200909234957463722150509502352, 318656154017908428228417947169274644685722918917857069783764522013541324145336, 622495459968043579891285967488587215780698872323739198100999331699121407526528, 482431239242047368856906785697843663156956380987420798775780793711446340994720, 178558471113770246205457738203197549502838833616954306069399584750786103108616, 525438976213709939171189072669733090118298956750810819126534009307462631386488, 431671514823488921765698022859678537009342717253730631112454427243049399248656, 623391136746194401589317793165440448623683548449718391408081324688728654464552, 47735035068247239017305611626898406900099175835847137940731385249827002652800, 561992016607456156836604032542824047616589253965985018703555367363655147584000, 562458230210967899918657026405534186438972802925858865484511934141982731208520, 593049243743310691011633282814531194843688098927075103536857943660458617354992, 492741529705296341446953293355848011101164733906804218248980709514572614190952, 349503415290501593296442714866925239630420603243469567749189589001228344226288, 598739059578243771880478378695238684926808417850857732567192556589068251289952, 374784764412054292131759549092077506345004554748744370059355739773049844638488, 386731334067244218705978768286954947023015456153227694283510834071964189361824, 253086706856836172905770649030004879466206640735747608454631212267175671005600, 528071209643963173369652151569817063976669322252192342749339151547313453633408, 149409982866808155626669022029853192971891647288527344832623547773148525980776, 69568705679834729189779361521391860137336472031234926165188092187485131155168, 562329737217617610271264096645408869787091489653669309087307070049957429815792, 423185504698532805754501159323218788336143856939054627298392212343404502798448, 84022654922133361656738617794996856733374963401696440156074917641246631417008, 597632738790113776470890332381586307635637171046702942397276146726526399985904, 312968212124830659635082483892516365585900871040120733402245067342582915324960, 249972977095662753658972300685628069019786065455057177229542677218350642539832, 46783301784255076315214395750265717906096022906466747031299548034817062517856, 476212784509904733158292758464487624774033308897836130892039553445442665356584, 491053466137442735426243563140146045789028014307983039975210204157274897857136, 499453616191397695163970054854851037200381316111240561271774357379081704869448, 450510959159015039632191670794732661486499182043449172265526792848272751635312, 495534352742109507293743495214200785775418358421745090208665545986866472078224, 307205372428582916520650654128815491309984367088227708010738832722528084018560, 362677470701858317330953098972713990451994186220040399227693557164705913858224, 348504104289368046519773801362497553286966911812215714035950697151914056349520, 403567806414051046395751156555299198065165058024301531419968859583448603116336, 178738215831475331738074101024102936836885295601244765439680479947215262075240, 656607279310380129425727437407154964239274906160027432548869045340705526890592, 405109097363929366652881385254616070877403581883103801593912560406999982439488, 162122473608124164327314277019757550124296553018118538033651263038023826074840, 539900967947844812694831206692740829293404262297630653077265804288290453373448, 597858835370528965757142468105253020629253019091353197279004011968819323075136, 388209584293086575850534842659942689439409850229692226908548990398349566377904, 533545728052078188920833933040542879638934929158899029309040027020484191623552, 292749586007204965933252178255012445518397772539436841511711519091619048106352, 447040031408901790963281766778523433063720302070483788606694016058020364068552, 663641533447117641782185144379897233521603341228227716592373202228057604891472, 177959167799187489926875370360644364070801488407044218809314738034856725159208, 371223849743746375586719787530350872087717577981222214908501312040937264291096, 406264837779912118146123123463228696003622726538594988698355109815182884603624, 151845332924847031833909505656025120238649734604022341636479834748838639981112, 284309583421675833328218548598799000687668246911636049390042761024614383867672, 93229441847023566739897524866648507126802080214767916270032671821267384708808, 615185570241652999556634448206997134213679996800676797774539938603252777562120, 9749586143140741841768449539318656723207207658356463937109501105358291048024, 221657651969498509053005570956040704259887583219073635021483855930144189219392, 29029092140243977789199492533651847559422852484353684103701588592260228946040, 23621784272669839633933114494921303583780285947760450859014432261831392892600, 156957260473772065627755736952147019244290381840357479420304261361088767314424, 310423440096632549539022220351433850977931713504175035225504130280328481121952, 25559084522035146864372196207427276738637059564972812037833513308842384454560, 418879281477855507919239087304332753243118633129065219113671198394623086033992, 428576525585017624681266719139526607494176078935917130102458309642712751672992, 364590282594864967775516051984904330256184262506554575910411956615286813755360, 5003864540093048182738838989370096737679195415890929130816640853824869908384, 389890412051964603110570979671316840386130409158607058950461342626808470594536, 422349848498497966065026442361310305174760534607375206136918070217875004343272, 137593688648863317007953083747566750472351279032585072432162870017379702246400, 625218080576658316717778234032781712071046655560037059910599523137745292678832, 444230778432348603357531519857175114194123097606832976924501004723535149421512, 521858533645297830962689745666792812886977922903507478589854957565622374361440, 403086681877210492352165149992549649153041585459767146321865513761994597582736, 212003959517887585279971927280643031926940345525596663360383235524011345905152, 295195149981181767259609499327394418121521342519528086013859242224834950133056, 438659718984135995934900882240406802472575259597343277862575287778939914833064, 510084036225806402499532152888832102494793498361823078025545764599367500634656, 195760673176229067260006167213428966673744041739165372136248842904878431692040, 13073784105630939265851804736196917169863180868814007429103171690190922002416, 563635930030990677160215424174432033831074835032913796548138156498778421773768, 600723226812822550916636314866258505900085408069367323619172150931060095914016, 14550100483490882794258451138759135570107011142807520527916602169588667617032, 405503714115068069017851724247044369700038388369690652259352554727070539144224, 569578868560794240944832342924507740286609303840127345984163402971953605793224, 442441245757537918163881266761849928873951017174569847361365180613733347652496, 341828542328500881590519096314277667948600397492396372225968000869441770788232, 639044927194186988834580958748231141941534868284934808029310305189602383452576, 654475074944901220679335850652483582281623175427904639173716411402878448217736, 154683529479331909027005471384814101669574663465829840863679078328674884348952, 568598512888974783039128095251102065395730201335179669792725279640461658971760, 94869164506164852189303737683635924761703699493444671691781214617609239723688, 329886026740475052792361375836533241133351589014379870060403210541783121534048, 82088554028302304274201053214464886978182713840293609578045457741294622639920, 633987044996761208628397235033672238201646243523135518650358805526563369526368, 317827600610540743463198277666662618498225307282164247748910072784398543806888, 609781108452774491125990739637888564294848728086795100470677571661990348437000, 448972483029908328465341369094172408454305177887107439863494949660802380403632, 334652477872319104407955841407658565844051990856246288744605130511467321715736, 587634786751628224341329339833849168574254795742529996430337692093623198343848, 42677480953872968546628001078999511116382464439780177334873981620146765114264, 494246715943097893333500860958725628727367681516315150769554428380554843923824, 581415156914206131365624356219615739001293978900774860869439791876610525486344, 161889708889677005883621919800872544382466038511514913265845977840963146807376, 236053118584357901405572618357001085591405152771912287755116725656921431099152, 497097842095962571303462734434761067683111725967163525797559748129038067427112, 447328875146227713574012340360687259574499413820014670217311140689470195331392, 600512916694579036418466397926742145264231846588644956733803108316262087954704, 516418371422028895063958282720515179405415960581042676615189809816748809071816, 564953818104315700094696757108203608316547115552076066234336470943810800563856, 355744587228626434307860431252389584091696960168734960788263116046675077020624, 495771690933573812644159357952714938276684510683962689458693385417849327960320, 504447309681131231807752864350009778841283886800364658606277941308701292097472, 634817952399582220490326366773550659846784263012944014466075677939970335427952, 514022199614205846102694153494994437834323348696420468735014345470146812591752, 42741740607608153702428072286070582883352394717490747018153415719930830642344, 528956247222579751450079167913924506960112161346959777231221456197513948833032, 317210545848921499610261293107643370004062992209155524520714664218412377139024, 75932599000104342026427394008136178723167001871832544680135462132937874660760, 338818763362743181761566268425874510514041395881236862895816982549580880982424, 130370013611862750949605458272515685086188815166649912636099400957919783933152, 166712562642883922367799420601259999768374273488810099910831683090341011195096, 287073461974364432229925049354224387453137471625908242272921470923054101873848, 501701985702559858401560405592004843531048823780721930698856643219004691412376, 529545194612319293547535738469870172944125158084436613456405661781722645735592, 0, 83294234028247848311169686881628090176722490540120450167841115859379627111273]]
#R = LLL_usual_condition_using_triangular(matrix(C), 0.75)
#print(R[:4])
n = 65537
#C = [[8*n,0,0,0,0],[0,32*n,0,0,0],[0,0,64*n,0,0],
#            [8*(-12), 32*7, 64*11, 1, 0],
#            [8*4, 32*(-9), 64*(-5), 0, n]]
R = LLL_usual_condition_on_triangular(matrix(C), 0.99)
print(R[:4])


[666353872225982786489357495053024721413779924320963601342728926875037016890184                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                                              0                                                    

In [4]:
R[0]

(666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

In [29]:
R[35]

(-232043514422508840699615328405130067366823482794108569564424205014822502466940207607285863610747226947185261247074041763205689034011528, 5888088699169368374069728815620119862633210562540627326416035306804218788447654832224471582531568092861271271942073494308872142233272, 74682513184412498359494872075989750929539890492090865518688082450111797554984299262339965097299171262122814530399583865393438341100080, -495798213136387490418213571636251306501166005536767643907910298889316478342556827145664208765506275431502190866013142505047996180084776, -168567342397034091237146767601225859333483049737116542885256088671018532565590471890923436636800030802308817943230166009187516758983984, -573039767947662133843764351867405974153487787975371442625474673505883189528777720602784492448836794470295417773373441786206587861381704, -864139128659276216411951599289528736295664089255043961518789599911259552281937833096836486776783339620715938602568605924552901271017656, 83727833613269508170616330012246143256

In [17]:
for i in range(60, 120):
    print(R[i])
    print()

(0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 666353872225982786489357495053024721413779924320963601342728926875037016890184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0)

(-53199369550360096290574853006046539303722581559785151007275837924580414879223726665990976535665517129551118188589174249344, -32656372660576857199758256679029172446824844608643484172828519921221480510935211880254755087403285770466637728184694182320, 17205819790411886182265695765170277388802507962360775064785689655963136247726074607583956960353548408331033182191888310280, -198728443677030989717939025911855978536078663294393871164824224133475907261681782128697770554707585837187689253236791054320, -49127386633256981072155851480

In [18]:
R.nrows()

130